# TDEsAsciiMetric


This notebook demonstrates how to use TDEsAsciiMetric which calculate what fraction of an input TDE lightcurve would be detected. Paired with a spatial slicer, you can obtain the skymap or the observed light curve. 


In [1]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pylab as plt

import healpy as hp
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db
import pandas as pd


In [2]:
# import metric from mafContrib
from mafContrib import TDEsAsciiMetric

## Simulated TDE light curve and the minimum requirement

The TDEsAsciiMetric provides some prameters to meet the requirement of detection TDEs.

- nObsTotal: Required total number of observations in each band.
- nObsPrePeak: Required Number of observations before peak.
- nObsNearPeak: Required number of observations in each band near peak.
- nFiltersNearPeak: Required number of filters near peak.
- nObsPostPeak: Required number of observations after peak.
- nFiltersPostPeak: Required number of number of filters after peak.

**Minimum requirement for detection TDEs** proposed by [Sjoert van Velzen](https://github.com/sjoertvv)

- one detection before peak in any band to make sure we can roughly resolve the time/flux at peak,  set nObsPrePeak=1;
- detections in three different bands within 10 days of peak to measure the color at peak, set nFiltersNearPeak=3, and nearPeakT=10;
- detections at least two bands post peak within two weeks to measure the change of color, set nFiltersPostPeak=2, and postPeakT=14.


**set parameters according to input lightcurve template and minimum requirement**

| parameters | value | meaning| 
| ---        | ---   |
| epochStart | -22   |  Start epoch of input light curve |
| peakEpoch  | 0     |  Peak epoch of input light curve  |
| nearPeakT  | 10    |  Within 10 days around peakEpoch are considered as near peak |
| postPeakT  | 14    |  Within two weeks after peakEpoch are considered as post peak |
| nPhaseCheck| 1     |  Number of phases, 1 means only check the same phase as in ascii file |
| nObsTotal  |{'u': 0, ...}| Number of total observations in each band |
| nObsPrePeak| 1     | Number of observations before peak |
|nObsNearPeak |{'u': 0, ...}| Number of observations in each band |
|nFiltersNearPeak | 3 | Number of filters near peak |
|nObsPostPeak | 0 | Number of observations after peak |
|nFiltersPostPeak | 2 | Number of filters after peak |

The lightcurve template is stored in an ascii file [TDEfaintfast_z0.1.dat](https://github.com/xiaolng/maf/blob/master/TDEfaintfast_z0.1.dat).

In [3]:
# light curve file
asciifile = 'TDEfaintfast_z0.1.dat'

# query columns 
mjdCol = 'observationStartMJD'
m5Col = 'fiveSigmaDepth'
filterCol = 'filter'
detectSNR={'u': 5, 'g': 5, 'r': 5, 'i': 5, 'z': 5, 'y': 5}

# light curve parameters
epochStart = -22
peakEpoch = 0
nearPeakT = 10
postPeakT = 14 # two weeks
nPhaseCheck = 1

# condition parameters
nObsTotal = {'u': 0, 'g': 0, 'r': 0, 'i': 0, 'z': 0, 'y': 0}
nObsPrePeak = 1
nObsNearPeak = {'u': 0, 'g': 0, 'r': 0, 'i': 0, 'z': 0, 'y': 0}
nFiltersNearPeak = 3
nObsPostPeak = 0 
nFiltersPostPeak = 2


Let's plot the light curve template. 

In [4]:
# plot light curve
def plotlc(dataSlice, time_key='time', mag_key='mag', filter_key='filter', err_key = None,
          peakEpoch=None, nearPeakT=None, postPeakT=None):
    # Input: dataSlice, pandas DataFrame or numpy ndarray
    
    colors = {'u':'k', 'g':'b', 'r':'r', 'i':'m', 'z':'orange', 'y':'c'}
    
    plt.figure(figsize=(10, 6))
    for f in np.unique(dataSlice[filter_key]):
        fmatch = dataSlice[filter_key]==f
        time = dataSlice[time_key][fmatch]
        mag = dataSlice[mag_key][fmatch]
        
        # draw a vertical line to denote pre/near/post peak
        if peakEpoch!=None:
            plt.axvline(x=peakEpoch, linestyle='-', linewidth=1)
            plt.axvline(x=peakEpoch-nearPeakT/2, linestyle='--', linewidth=1)
            plt.axvline(x=peakEpoch+nearPeakT/2, linestyle='--', linewidth=1)
            plt.axvline(x=peakEpoch+nearPeakT/2+postPeakT, linestyle='--', linewidth=1)
        
        if err_key!=None:
            mag_err = dataSlice[err_key][fmatch]
            plt.errorbar(time, mag, yerr=mag_err, color=colors[f], fmt='.', label=f)
        else:
            plt.scatter(time, mag, color=colors[f], marker='*', label=f)

    plt.legend()
    plt.xlabel('days')
    plt.ylabel('mag')
    plt.ylim(plt.ylim()[::-1])

In [5]:
asciifile = '../../data/tde/TDEfaintfast_z0.1.dat'

lcv_template = np.genfromtxt(asciifile, dtype=[('ph', 'f8'), ('mag', 'f8'), ('flt', 'S1')])

df = pd.DataFrame()
df['ph'] = lcv_template['ph']
df['mag'] = lcv_template['mag']
df['flt'] = lcv_template['flt'].astype(str)

plotlc(df, time_key='ph', mag_key='mag', filter_key='flt', 
       peakEpoch=peakEpoch, nearPeakT=nearPeakT, postPeakT=postPeakT)

plt.xlabel('epoch [days] \n An example of an input lightcurve template. \n' )

Text(0.5, 0, 'epoch [days] \n An example of an input lightcurve template. \n')

## Opsim database

Connect to opsim database and get the proposalId

In [7]:
# connect to opsim database
opsdb_baseline = db.OpsimDatabase('baseline_v1.3_10yrs.db')
opsdb_baseline.fetchPropInfo()

No proposal table available - no proposalIds have been assigned.


({}, {})

In [8]:
# output directory
outDir = 'outdir'
resultsDb = db.ResultsDb(outDir=outDir)


### get skymap

In [9]:
# get skymap by set dataout=False and use HealpixSlicer()
transmetric = TDEsAsciiMetric(asciifile=None, mjdCol=mjdCol, m5Col=m5Col, filterCol=filterCol, 
                              detectSNR=detectSNR, epochStart=epochStart, peakEpoch=peakEpoch, 
                              nearPeakT=nearPeakT, postPeakT=postPeakT, nPhaseCheck=nPhaseCheck,
                              nObsTotal= nObsTotal, nObsPrePeak=nObsPrePeak, 
                              nObsNearPeak=nObsNearPeak, nFiltersNearPeak=nFiltersNearPeak, 
                              nObsPostPeak=nObsPostPeak, nFiltersPostPeak=nFiltersPostPeak,
                              dataout=False)

slicer = slicers.HealpixSlicer(nside=16) 
sqlconstraint = 'night<730'

transmetricSky = metricBundles.MetricBundle(transmetric,slicer,sqlconstraint)

group = metricBundles.MetricBundleGroup({'transmetricSky':transmetricSky}, opsdb_baseline, outDir=outDir, resultsDb=resultsDb)
group.runAll()
group.plotAll(closefigs=False)

Finish initializing metric
Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint night<730 for columns ['fieldRA', 'observationStartMJD', 'fieldDec', 'filter', 'fiveSigmaDepth']
Found 420180 visits
Running:  ['transmetricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Plotting figures with "night<730" constraint now.
monopole: 0.0419786  dipole: lon: -9.38153, lat: -67.2205, amp: 0.0366427
Plotting complete.


We can create a pd.DataFrame and save the skymap info for later use.

In [10]:
baseline2018_skymap = transmetricSky.metricValues

# create a dataframe and save to csv file
df = pd.DataFrame(baseline2018_skymap)
df['ipix'] = df.index 
df['ra'] = hp.pix2ang(ipix=df['ipix'], nside=16, lonlat=True)[0]
df['dec'] = hp.pix2ang(ipix=df['ipix'], nside=16, lonlat=True)[1]
df['metricValues'] = df[0]
del df[0]
df.to_csv('./opsdb/baseline2018_skymap.csv', index=False, na_rep='NaN')

df.head()

FileNotFoundError: [Errno 2] No such file or directory: './opsdb/baseline2018_skymap.csv'

In [ ]:
# view all metric values 
np.unique(baseline2018_skymap)

In [ ]:
# get the all ipixes for a metric value
np.where(baseline2018_skymap==0.2) 

In [ ]:
# get the (ra,dec) for a ipix
hp.pix2ang(ipix=1549, nside=16, lonlat=True)

### get light curve 

In [ ]:
## get light curve by set dataout=True and use UserPointSlicer()
transmetric = TDEsAsciiMetric(asciifile=asciifile, mjdCol=mjdCol, m5Col=m5Col, filterCol=filterCol, detectSNR=detectSNR,
                 epochStart=epochStart, peakEpoch=peakEpoch, nearPeakT=nearPeakT, nPhaseCheck=nPhaseCheck,
                 nObsTotal= nObsTotal,nObsPrePeak=nObsPrePeak, 
                 nObsNearPeak=nObsNearPeak, nFiltersNearPeak=nFiltersNearPeak, 
                 nObsPostPeak=nObsPostPeak, nFiltersPostPeak=nFiltersPostPeak,
                 dataout=True)
# ra,dec of ipix 
ra = np.array([255.9375])
dec = np.array([0.0])
slicer = slicers.UserPointsSlicer(ra, dec)

sqlconstraint = 'night<730 and proposalId=3'
transmetricSky = metricBundles.MetricBundle(transmetric,slicer,sqlconstraint)

group = metricBundles.MetricBundleGroup({'transmetricSky':transmetricSky}, opsdb_baseline, outDir=outDir, resultsDb=resultsDb)
group.runAll()


The output metricValues is a list of dictionaries for each phase(tshift). Each dictionary contains arrays of 

|  variable   |  meaning   |
| --- | --- |
| 'tshift' | the phase shift of the light curve|
| 'expMJD' | the time column |
| 'm5'     | the fiveSigmaDepth |
|'filters' | filters |
| 'lcNumber'| number of light curve|
|'lcEpoch' | epoch of the light curve |
| 'prePeakCheck' | equals True for prePeak observations |
| 'nearPeakCheck'| equals True for nearPeak observations|
| 'postPeakCheck'| equals True for postPeak observations|
| 'lcMags'| magnitudes of the light curve|
| 'lcSNR' | signal to noise ratio|
| 'lcMagsStd'| standard deviation of magnitudes|
| 'lcAboveThresh'| equals True if for detected magnitudes|
| 'detected' |equals True if this light curve was detected|

In [ ]:
# save file and plot light curve
output_dict_list = transmetricSky.metricValues.data[0]
df = pd.DataFrame(output_dict_list[0])

df.to_csv('./opsdb/baseline2018_lc.csv', index=False)

df.head()

The output light curve can be plotted use the same function defined above.

In [ ]:
# load from csv
# df = pd.read_csv('./opsdb/baseline2018_lc.csv')

# plot all light curve
# plotlc(df[ (df['lcAboveThresh'])], time_key='expMJD', mag_key='lcMags', filter_key='filters', err_key='lcMagsStd')

# plot a specific light curve
plotlc(df[ (df['lcAboveThresh']) & (df['lcNumber']==0)], time_key='lcEpoch', mag_key='lcMags', filter_key='filters', err_key='lcMagsStd', 
       peakEpoch=peakEpoch, nearPeakT=nearPeakT, postPeakT=postPeakT)

plt.xlabel('epoch [days] \n An example of detected light curve from baseline2018.\n' + 
           'It meets the minimum requirements: one detection before peak,\n three filters near peak, and two filters post peak within two weeks ')



Other opsim databases can be explored by following the same procedure.